In [4]:
import gspread
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from datetime import datetime
from openai import OpenAI
import os
import json
from dotenv import load_dotenv
from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, filters, ContextTypes, Application
import requests
import logging

In [5]:
load_dotenv()
telegram_bot_api_key = os.getenv('TELEGRAM_BOT_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')
OpenAI.api_key = openai_api_key
client = OpenAI()

In [7]:
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)
# set higher logging level for httpx to avoid all GET and POST requests being logged
logging.getLogger("httpx").setLevel(logging.WARNING)

logger = logging.getLogger(__name__)

In [8]:
SCOPE = [
    "https://www.googleapis.com/auth/spreadsheets"
    ]

CREDS = Credentials.from_service_account_file('creds.json', scopes=SCOPE)
# SCOPED_CREDS = CREDS.with_scopes(SCOPE)
GSPREAD_CLIENT = gspread.authorize(CREDS)
SHEET_ID = "1V8tqqiW1AGPfl9FH1HPZ_ko0wVNJFakmfEHCKak01Xo"
SHEET = GSPREAD_CLIENT.open_by_key(SHEET_ID)
SERVICE = build('sheets', 'v4', credentials=CREDS)

2025-01-02 16:18:38,112 - googleapiclient.discovery_cache - INFO - file_cache is only supported with oauth2client<4.0.0


In [9]:
class DataImporter:
    def import_data(self, json_data):
        # Ensure json_data is a dictionary
        if isinstance(json_data, str):
            json_data = json.loads(json_data)
        date = json_data['date']
        amount = json_data['amount']
        currency = json_data['currency']
        trans_type = json_data['trans_type']
        category = json_data['category']
        note = json_data['note']
        account = json_data['account']
        values = [date, amount, currency, trans_type, category, note, account]
        return values

    def append_data_to_last_row(self, sheet, data):
        """
        Appends data to the last row of a Google Sheet.

        Args:
            sheet (gspread.Worksheet): The Google Sheet to append data to.
            data (list): A list of data to append to the last row.
        """
        # Call the Sheets API to append data
        response = sheet.append_row(values=data)
        return response

In [10]:
prompt = """
    You are a helpful assistant that helps users to manage their finances in 2025. You will extract data from user notes by both English and Vietnamese.
    The user will input a transaction note in Vietnamese or English. You will extract data from the note and return the data in a JSON format, each field contains only one value.
    The JSON format should be:
    {
        "date": dd/mm/2025,
        "amount": number,
        "currency": "VND, USD",
        "trans_type": "Thu, Chi, Đổi",
        "category": "Ăn uống, Giải trí, Hóa đơn, Personal, Quà, Thu coin, Trả ví, Thu code, Trả code, Trả lương, Lương",
        "note": "note",
        "account": "VP, Momo, Paypal, Binance, Techcombank, VCB, Tsr"
    }
    
    With not related input, just return "Invalid input"
"""

user_prompt = "2/1 573k thu tiền coin nạp game zalo VP"

In [11]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": prompt},
        {
            "role": "user",
            "content": user_prompt
        }
    ]
)

In [12]:
json_data = completion.choices[0].message.content
print(json_data)

{
    "date": "02/01/2025",
    "amount": 573000,
    "currency": "VND",
    "trans_type": "Thu",
    "category": "Thu coin",
    "note": "nạp game",
    "account": "VP"
}


In [13]:
details_sheet = SHEET.worksheet("Details")
data_importer = DataImporter()
values = data_importer.import_data(json_data)
response = data_importer.append_data_to_last_row(details_sheet, values)
print(response)

{'spreadsheetId': '1V8tqqiW1AGPfl9FH1HPZ_ko0wVNJFakmfEHCKak01Xo', 'tableRange': 'Details!A1:G11', 'updates': {'spreadsheetId': '1V8tqqiW1AGPfl9FH1HPZ_ko0wVNJFakmfEHCKak01Xo', 'updatedRange': 'Details!A12:G12', 'updatedRows': 1, 'updatedColumns': 7, 'updatedCells': 7}}
